In [1]:
import pandas as pd
import numpy as np
import random
import time
import itertools

C:\Users\Amanda\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Letter Mapping

In [2]:
# Define the letter maps for each problem
letter_maps = {
    "SEND+MORE=MONEY": {'S': 0, 'E': 1, 'N': 2, 'D': 3, 'M': 4, 'O': 5, 'R': 6, 'Y': 7},
    "EAT+THAT=APPLE": {'E': 0, 'A': 1, 'T': 2, 'H': 3, 'P': 4, 'L': 5},
    "CROSS+ROADS=DANGER": {'C': 0, 'R': 1, 'O': 2, 'S': 3, 'A': 4, 'D': 5, 'N': 6, 'G': 7, 'E': 8},
    "COCA+COLA=OASIS": {'C': 0, 'O': 1, 'A': 2, 'L': 3, 'S': 4, 'I': 5},
    "DONALD+GERALD=ROBERT": {'D': 0, 'O': 1, 'N': 2, 'A': 3, 'L': 4, 'G': 5, 'E': 6, 'R': 7, 'B': 8, 'T': 9}
}




## Pipeline

In [3]:
# Convert individual's digit mapping to a dictionary that maps letters to digits
def get_mapping(individual, letter_map):
    return {letter: individual[i] for i, letter in enumerate(letter_map)}

# Convert a word into its numeric value using the mapping
def word_to_num(word, mapping):
    return int(''.join(str(mapping[letter]) for letter in word))

# Função de fitness para um problema específico
def fitness_minimize_diff(individual, equation, letter_map):
    mapping = get_mapping(individual, letter_map)

    left_side, right_side = equation.split('=')
    left_words = left_side.split('+')
    
    left_sum = sum(word_to_num(word, mapping) for word in left_words)
    right_sum = word_to_num(right_side, mapping)
    
    return abs(left_sum - right_sum)

def get_mapping_digit(individual, letter_map):
    # Cria um mapeamento de letras para dígitos no indivíduo
    return {letter: digit for letter, digit in zip(letter_map.keys(), individual)}

def fitness_digit_by_digit(individual, equation, letter_map):
    # Cria o mapeamento de letras para dígitos
    mapping = get_mapping_digit(individual, letter_map)
    
    # Divide a equação entre lado esquerdo e direito
    left_side, right_side = equation.split('=')
    left_words = left_side.split('+')

    # Cálculo das somas dos dois lados da equação
    left_sum = sum(word_to_num(word, mapping) for word in left_words)
    right_sum = word_to_num(right_side, mapping)

    # Verificação de zeros à esquerda
    if any(mapping[word[0]] == 0 for word in left_words + [right_side]):
        return float('inf')  # Solução inválida, retorna a pior avaliação possível

    # Garantir que ambos os números tenham o mesmo tamanho para a comparação dígito a dígito
    max_len = max(len(str(left_sum)), len(str(right_sum)))
    left_digits = str(left_sum).zfill(max_len)
    right_digits = str(right_sum).zfill(max_len)

    # Função de avaliação baseada na diferença proporcional dos dígitos
    fitness_penalty = 0
    for i, (ld, rd) in enumerate(zip(left_digits, right_digits)):
        # Quanto mais à esquerda o dígito, maior o peso da penalidade
        weight = 10 ** (len(left_digits) - i - 1)
        fitness_penalty += abs(int(ld) - int(rd)) * weight

    return fitness_penalty

def tournament_selection(population, fitnesses, k=3):
    selected = random.sample(list(zip(population, fitnesses)), k)
    return min(selected, key=lambda x: x[1])[0]

def roleta_selection(population, fitnesses):
    total_fitness = sum(1.0 / f for f in fitnesses if f > 0)  # Somente fitness válidos (não infinitos)
    selection_probs = [(1.0 / f) / total_fitness for f in fitnesses if f > 0]
    
    # Selecionar o par de pais
    parent1 = random.choices(population, weights=selection_probs, k=1)[0]
    parent2 = random.choices(population, weights=selection_probs, k=1)[0]
    
    return parent1, parent2

def roleta_selection(population, fitnesses):
    # Convert fitnesses to a form where higher fitness means higher chance of selection
    total_fitness = sum(fitnesses)
    selection_probs = [f / total_fitness for f in fitnesses]  # Proportion of each fitness
    
    # Select two parents
    parent1 = random.choices(population, weights=selection_probs, k=1)[0]
    parent2 = random.choices(population, weights=selection_probs, k=1)[0]
    
    return parent1, parent2
#################### CROSSOVER
#Função de correção para remover repetições
def fix_repetitions(individual):
    missing_values = [i for i in range(10) if i not in individual]
    duplicates = {x for x in individual if individual.count(x) > 1}
    for i, value in enumerate(individual):
        if value in duplicates:
            individual[i] = missing_values.pop(0)
            duplicates.remove(value)
    return individual

# Crossover Cíclico adaptado
def cyclic_crossover_adaptado(parent1, parent2):
    size = len(parent1)
    child1 = [-1] * size
    child2 = [-1] * size
    cycle_indices = []
    idx = 0
    
    while idx not in cycle_indices:
        cycle_indices.append(idx)
        child1[idx] = parent1[idx]
        child2[idx] = parent2[idx]
        next_value = parent2[idx]
        
        if next_value not in parent1:
            break

        idx = parent1.index(next_value)
    
    for i in range(size):
        if i not in cycle_indices:
            child1[i] = parent2[i]
            child2[i] = parent1[i]

    # Correção para garantir filhos válidos (sem repetições)
    if len(set(child1)) != len(child1):
        child1 = fix_repetitions(child1)
    if len(set(child2)) != len(child2):
        child2 = fix_repetitions(child2)

    return child1, child2

def pmx_crossover(parent1, parent2):
    size = len(parent1)
    
    # Inicializa filhos com valores inválidos (-1)
    child1 = [-1] * size
    child2 = [-1] * size
    
    # Seleciona dois pontos de crossover
    cx_point1, cx_point2 = sorted(random.sample(range(size), 2))

    # Troca segmentos entre os pais
    child1[cx_point1:cx_point2] = parent2[cx_point1:cx_point2]
    child2[cx_point1:cx_point2] = parent1[cx_point1:cx_point2]

    def fill_remaining(child, parent, cx_point1, cx_point2):
        # Preenche os valores restantes do filho
        for i in range(size):
            if cx_point1 <= i < cx_point2:
                continue
            if child[i] == -1:
                value = parent[i]
                while value in child:
                    value = parent[child.index(value)]
                child[i] = value
    
    fill_remaining(child1, parent1, cx_point1, cx_point2)
    fill_remaining(child2, parent2, cx_point1, cx_point2)
    
    return child1, child2


#################### Função de mutação (troca de valor entre 2 posições)
def mutate(individual):
    idx1, idx2 = random.sample(range(len(individual)), 2)
    individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

# Reinserção ordenada (melhores entre pais e filhos)
def reinsercao_ordenada(population, fitnesses, new_population, new_fitnesses):
    combined_population = population + new_population
    combined_fitnesses = fitnesses + new_fitnesses
    sorted_population = [x for _, x in sorted(zip(combined_fitnesses, combined_population))]
    return sorted_population[:len(population)]  # Retorna o número correto de indivíduos

# Reinserção pura com elitismo de 20%
def reinsercao_pura_com_elitismo(population, fitnesses, new_population, new_fitnesses, elitism_rate=0.2):
    population_size = len(population)
    num_elite = int(elitism_rate * population_size)

    sorted_population = [x for _, x in sorted(zip(fitnesses, population))]
    elite = sorted_population[:num_elite]  # Seleciona os 20% melhores

    remaining_population = random.sample(new_population, population_size - num_elite)
    return elite + remaining_population

# Criação da população inicial para um problema específico
def create_initial_population(size, letter_map):
    population = []
    num_letters = len(letter_map)
    
    for _ in range(size):
            individual = random.sample(range(10), 10)  # Unique digits for the letters in the map
            population.append(individual)
            # Ensure that non-zero letters do not map to zero
    return population
#################### Algoritmo Genético
# Algoritmo Genético
def genetic_algorithm(population_size=100, num_generations=100, crossover_rate=0.8, mutation_rate=0.1, selection_method='', cross_over='', reinsercao='ordenada', letter_map='', problem=''):
    population = create_initial_population(population_size, letter_map)


    for generation in range(num_generations):
        fitnesses = [fitness_digit_by_digit(ind, problem, letter_map) for ind in population]
        
        if 0 in fitnesses:
            solution = population[fitnesses.index(0)]
            return fitnesses, solution

        new_population = []
        new_fitnesses = []

        while len(new_population) < population_size:
            # Seleção dos pares de pais
            if selection_method == 'torneio_3':
                parent1 = tournament_selection(population, fitnesses)
                parent2 = tournament_selection(population, fitnesses)
            elif selection_method == 'roleta':
                parent1, parent2 = roleta_selection(population, fitnesses)

            # Crossover
            if random.random() < crossover_rate:
                if cross_over == 'cross_ciclico':
                    child1, child2 = cyclic_crossover_adaptado(parent1, parent2)
                elif cross_over == 'cross_pmx':
                    child1, child2 = pmx_crossover(parent1, parent2)
            else:
                child1, child2 = parent1[:], parent2[:]

            if random.random() < mutation_rate:
                mutate(child1)
            if random.random() < mutation_rate:
                mutate(child2)

            new_population.extend([child1, child2])
            new_fitnesses.extend([fitness_digit_by_digit(child1, problem, letter_map), fitness_digit_by_digit(child2, problem, letter_map)])

        if reinsercao == 'ordenada':
            population = reinsercao_ordenada(population, fitnesses, new_population, new_fitnesses)
        else:
            population = reinsercao_pura_com_elitismo(population, fitnesses, new_population, new_fitnesses)

    fitnesses = [fitness_digit_by_digit(ind, problem, letter_map) for ind in population]
    return fitnesses, min(population, key=lambda ind: fitness_digit_by_digit(ind, problem, letter_map))



# Função para executar os testes 100 vezes
def run_tests(num_tests=100, population_size=100, num_generations=100, crossover_rate=0.8, mutation_rate=0.1, selection_method='', cross_over='', reinsercao='R1', letter_map='', problem=''):
    fitness_results = []
    convergence_count = 0
    execution_times = []

    for _ in range(num_tests):
        start_time = time.time()
        fitnesses, best_individual = genetic_algorithm(population_size, num_generations, crossover_rate, mutation_rate, selection_method, cross_over, reinsercao, letter_map, problem)
        execution_times.append(time.time() - start_time)

        fitness_results.append(min(fitnesses))
        if min(fitnesses) == 0:
            convergence_count += 1

    best_fitness = min(fitness_results)
    fitness_std_dev = np.std(fitness_results)
    avg_execution_time = np.mean(execution_times)
    convergence_rate = (convergence_count / num_tests) * 100

    return best_fitness, fitness_std_dev, avg_execution_time, convergence_rate




## Test Combinations

In [ ]:
# Define the parameter space
mutation_rates = [0.1, 0.2, 0.3]
selection_methods = ['torneio_3', 'roleta']
crossovers = ['cross_ciclico', 'cross_pmx']
reinsercoes = ['ordenada', 'elitismo']
num_generations = [50, 80, 100]
#problem = 'EAT+THAT=APPLE'
problem = 'SEND+MORE=MONEY'
#problem = 'CROSS+ROADS=DANGER'
#problem = 'COCA+COLA=OASIS'
#problem = 'DONALD+GERALD=ROBERT'
# Store the results
results = []

# Iterate through all combinations of parameters
for mutation_rate, selection_method, cross_over, reinsercao, n_generations in itertools.product(mutation_rates, selection_methods, crossovers, reinsercoes, num_generations):
    # Run the tests for the current configuration

    if reinsercao == 'elitismo':
        crossover_rate = 0.8
    else:
        crossover_rate = 0.6

    best_fitness, fitness_std_dev, avg_execution_time, convergence_rate = run_tests(
        num_tests=100,
        population_size=100,
        num_generations=n_generations,
        crossover_rate=crossover_rate,
        mutation_rate=mutation_rate,
        selection_method=selection_method,
        cross_over=cross_over,
        reinsercao=reinsercao,
        letter_map=letter_maps[problem],
        problem=problem
    )
    
    # Store the results for the current configuration
    results.append({
        'mutation_rate': mutation_rate,
        'num_generations': n_generations,
        'cross_over_rate': crossover_rate,
        'selection_method': selection_method,
        'cross_over': cross_over,
        'reinsercao': reinsercao,
        'best_fitness': best_fitness,
        'fitness_std_dev': fitness_std_dev,
        'avg_execution_time': avg_execution_time,
        'convergence_rate': convergence_rate
    })

#fitness2 é o digit by digit
#fitness1 é o apresentado em sala
# Create the DataFrame from the results
df_results = pd.DataFrame(results)
#df_results.to_csv(f'{problem}.csv', index=False)
df_results

## Test Best Config

In [5]:
# Define the parameter space
mutation_rates = [0.6]
selection_methods = ['roleta']
crossovers = ['cross_pmx']
reinsercoes = ['ordenada']
num_generations = [200]
num_population = [400]

problems=['EAT+THAT=APPLE', 'SEND+MORE=MONEY', 'CROSS+ROADS=DANGER',  'COCA+COLA=OASIS', 'DONALD+GERALD=ROBERT']
problems=['COCA+COLA=OASIS']

# Store the results
results = []

for problem in problems:

    # Iterate through all combinations of parameters
    for mutation_rate, selection_method, cross_over, reinsercao, n_generations, n_population in itertools.product(mutation_rates, selection_methods, crossovers, reinsercoes, num_generations, num_population):
        # Run the tests for the current configuration

        if reinsercao == 'elitismo':
            crossover_rate = 0.8
        else:
            crossover_rate = 0.6

        best_fitness, fitness_std_dev, avg_execution_time, convergence_rate = run_tests(
            num_tests=100,
            population_size=n_population,
            num_generations=n_generations,
            crossover_rate=crossover_rate,
            mutation_rate=mutation_rate,
            selection_method=selection_method,
            cross_over=cross_over,
            reinsercao=reinsercao,
            letter_map=letter_maps[problem],
            problem=problem
        )
        
        # Store the results for the current configuration
        results.append({
            'mutation_rate': mutation_rate,
            'population_size': n_population,
            'num_generations': n_generations,
            'cross_over_rate': crossover_rate,
            'selection_method': selection_method,
            'cross_over': cross_over,
            'reinsercao': reinsercao,
            'best_fitness': best_fitness,
            'fitness_std_dev': fitness_std_dev,
            'avg_execution_time': avg_execution_time,
            'convergence_rate': convergence_rate,
            'problem': problem
        })

df_results = pd.DataFrame(results)
#df_results.to_csv(f'etapa2.csv', index=False)
df_results

,mutation_rate,population_size,num_generations,cross_over_rate,selection_method,cross_over,reinsercao,best_fitness,fitness_std_dev,avg_execution_time,convergence_rate,problem
0,0.6,400,200,0.6,roleta,cross_pmx,ordenada,0,0.237487,0.866866,94.0,COCA+COLA=OASIS
